# 师生学习训练 se3 策略
单独训练 `inhand_se3_env_cfg.py` 非常困难，训练出的策略虽然转动物体，但不是通过指尖与物体的接触，而是指根指节等部位“顶”着物体转的，显然不是想要的结果。
<br>目前 `AnyRotate/logs/rl_games/leaphand_object_rot/2026-01-12_16-46-33` 等关节空间训练出来的策略，手指步态比较良好，想通过模仿学习的方式，训练出更好的 se3 策略。然后再尝试泛化到不同手型。手型的泛化是本工作的 **key contributions**。
<br>本笔记大部分都是背景和思考，明确的任务需求放在 **Plan** 部分

## RL设计
过去的 `inhand_se3_env_cfg.py` 在观察-动作-奖励设计的基础上打算在增添一些新的改进

### 观察空间
因为要考虑手型的泛化，所以在观察空间中不能加入关节量，因为每个手型的关节数量、上下限范围等都不一样。之前用的是 `body_twists` 代替关节量，但这个观察项很明显对于任务还是很稀缺，对于实机部署是不利的。为此结合最近的探索，又加入触觉信息。可参考 `inhand_tactile_env_cfg.py`。加1个0-1二值触觉观察信号

#### Problem
**背景情况：**
- 单独训练 `inhand_se3_env_cfg.py` 时发现空间坐标系 {s} 表现优于刚体坐标系 {b}
- {s} 的物理语义相对固定明确，而 {b} 是指尖坐标系会时变，导致旋量物理语义浮动性大
- 从泛化角度考虑，{b} 更有优势：无需考虑 {s} 的具体设置，有利于实机部署和跨手型迁移（如从LeapHand迁移到Allegro Hand）

**核心问题：**
如何在保持 {b} 坐标系泛化优势的前提下，提升SE(3)策略的训练表现？

**当前考虑的解决方案：**
1. **直接师生学习**：用 {s} 训练的teacher指导 {b} 的student，但担心单时间步 {b} 旋量信息歧义性太大
2. **时序编码方案**：参考Rapid Motor Adaptation思路，对过去数十个时间步的 {b} 旋量信息进行编码，结合触觉或特权信息作为观察输入，从时序中捕捉更丰富的信息。但是编码器采用什么架构？用什么方式训练？师生学习在线蒸馏时，同时优化编码器和se3策略模型吗？
3. **丰富观察信息**：加入指尖触觉（0-1信号值），这个目前可行；加入物体点云等更多指尖和物体的想对信息，或者直接加入RGB视觉等，但关于点云、RGB等视觉模态的信息目前没有实践，目前偏向后期探索完后再加入，作为底牌。

#### Plan
目前的观察空间构思是这样的：
- 本体感受（se3版): $Prop_t = \{\mathcal{V}_{real,t}, \mathcal{V}_{act,t}$，$\mathcal{V}_{real,t}\}$. 真实状态下的参考点在末端的旋量，$\mathcal{V}_{act,t}$ 是上一步动作的参考点在末端的旋量，两者皆是归一化的。计划采用 $\mathcal{V}_b$。类比于关节空间策略中本体感受是当前关节状态和上一步关节动作，但也是归一化的。
- 触觉信息：$\tau_t$. 每个指尖的0-1二值触觉信号，表示是否接触到物体。
- 潜在信息：$h_t = Encoder\{Prop_t, \tau_t\}_{t-N+1:t}$. 这个主要是为了缓解 $\mathcal{V}_{b}$ 旋量单时间步歧义性大的问题，通过时序编码器（TCN, RNN或Transformer）对过去N个时间步的本体感受和触觉信息进行编码，得到潜在信息 $h_t$，作为观察输入的一部分。
- 指令信息：$c_t$. 这个和关节空间策略一致，是物体目标位姿信息。
> 这里不包含物体位置等特权信息，因为这个是学生策略，要部署到真机上

### 动作空间
计划采用 se3dlsActionsCfg，这个是过去的实验中表现最好的。这里有1个疑惑，是否有必要再加入 EMA 平滑，就像关节空间中的EMA动作项？
$$\theta\left(t+1\right)=\theta(t)+J^{+}\left(\theta\left(t\right)\right)\mathcal{V}\left(t\right)\Delta{t}$$
> 注：这里的 $\mathcal{V}$ 是参考点在末端（b或b'），但参考系在 $\{s\}$，也可以选择 $\mathcal{V_{b}}$，参考点和参考系都在末端 $\{b\}$，这便和 $\{s\}$ 无关了，和全局坐标系也无关，理论上更具有泛化。但之前在 `inhand_se3_env_cfg.py` 就实际训练表现， $\mathcal{V_{s}}$ 要好一些。

从上面这个公式来看，因为 $\Delta{t}$ 本身很小，本身就对关节增量起到了一定的平滑作用。但这里的EMA，是对 $\mathcal{V_{s}}$ 进行平滑，而不是关节增量，所以还是有一定区别的。这里需要考虑一下，个人认为速度通常也是平滑变换的量，很少突变（当然这里因为是contact-rich task，所以会经常发送，但这个旋量是期望，不代表真实），可能也需要EMA平滑吧。

#### Plan
目前计划采用EMA，因为 $\Delta{t}$ 的平滑和 EMA 平滑是不同层面的。前者是在幅值上减小抖动的程度，后者是对策略输出的高频抖动平滑。计划设计1个新的动作空间配置类 `se3dlsEmaActionsCfg`，继承自 `se3dlsActionsCfg`，和新的动作项 `se3dlsEmaActions`，继承自 `se3dlsActions`。可参考关节空间中的 `EMAJointPositionToLimitsAction`

### 奖励

#### Plan
借鉴 `inhand_tactile_env_cfg.py`，补上和触觉有关的奖励项。

## 师生学习算法
这里借鉴 `learning.ipynb` 中 `Imitation Learning`部分。之前的思路是让关节空间训练好的策略 $\pi_E$ rollout来产生数据，然后通过 BC 或 Diffusion 等方法来训练 se3 策略。但关节空间和任务空间策略的观察和动作空间不一致，为此进行的数据工程十分繁琐，或许 `RL + DAgger` 是更合适与易实现些的方式。这种方式是不是相当于师生在线RL蒸馏？如果是的话，`/home/hac/isaac/DEXTRAH` 这个灵巧抓取项目也用了类似的师生在线RL蒸馏方法，可以参考一下他们的实现细节（稍微注意下它们是IsaacLab2.2.1/IsaacSim5.0.0，环境是 DirectRLEnv，我们这IsaacLab2.3.0/IsaacSim5.1.0，环境是 ManagerBasedRLEnv，可能相差不大但还是要注意）。
<br>如果采用 `RL + DAgger` 的方式，那么关键应该在于动作对齐：

**Joint-space policy**: $o_J \xrightarrow{\pi(\cdot)} a \xrightarrow{\text{affine}} \Delta\theta \xrightarrow{\theta_{cur}} \theta_{des} \xrightarrow{\text{clip}} \theta_{act}$

**se3-space policy**: $o_T \xrightarrow{\pi(\cdot)} a \xrightarrow{\text{affine}} \mathcal{V_{b'}} \xrightarrow{Ad_{T_{bb'}}} \mathcal{V_b} \xrightarrow{J_{dls}^+} \dot{\theta} \xrightarrow{\Delta t} \Delta\theta \xrightarrow{\text{clip}} \theta_{act}$

这里是对齐在 $\theta_{act}$ 处，也就是最终的执行关节位置。

### Plan
采用 `RL + DAgger` 的方式进行师生在线RL蒸馏,大致框架参考 `DEXTRAH` 中的实现，但部分细节可能需要根据我们当前的环境和需求进行调整。
关于监督损失，因为对齐是在 $\theta_{act}$ 处，所以可以直接使用关节位置的MSE损失作为监督信号，但反向梯度传播可能有一定影响，需要裁减。目前有两个想法，一个是reward shaping，把动作损失本身也作为奖励的一部分。另一个便是反向梯度传播更新 + 动作裁减。需要实验验证哪种方式更有效。

### 记录
1. `runs/leaphand_se3_tactile_rot_14-02-47-28/nn/leaphand_se3_tactile_rot.pth`
- 从训练效果本身来看，动作非常缓慢，几乎不动。训练到了5200epochs，奖励也只有-32,连正数都没达到。有可能是指数平滑系数比较大，但根本原因应该不是这个，训练失败可能有观察空间设计的问题，网络架构的问题等
- 还有一些小工程问题，回放脚本需要动play.py，不如在distillation目录处重新设计1个；师生策略训练时，训练的策略保存到了 `/home/hac/isaac/runs`，这不符合我的意愿，最好保存到 `/home/hac/isaac/AnyRotate/logs/rl_games` 下，方便管理，且子目录名称和学生策略的任务环境名一致，应为，如`AnyRotate/logs/rl_games/leaphand_se3_rot`这种